# Bundling Analysis

In [3]:
# libraries
library(tidyverse)
library(arules)
library(arulesViz)
library(RPostgreSQL)
library(tsibble)
library(gridExtra)
library(clipr)
library(stringr)
library(config)
library(googlesheets4)
library(jsonlite)

# setting wd
if ( grepl("bundles", getwd(), fixed = TRUE) ) {
    print("current working directory is fine")
} else {
    setwd("./projects/bundles")
    print("changed working directory")
}

[1] "current working directory is fine"


In [4]:
# load functions
source("bundle-functions.R")

### Get Transactions

In [5]:
# get credentials
dw <- config::get(file="./redshift_credentials.yml")

In [6]:
options(warn=-1)
# connecting to dw
con <- DBI::dbConnect(RPostgreSQL::PostgreSQL(),
                      dbname = dw$dbname,
                      host = dw$host,
                      port = dw$port,
                      user = dw$user,
                      password = dw$password)

In [7]:
# Returns string without leading or trailing white space
trim <- function (x) gsub("^\\s+|\\s+$", "", x)

In [17]:
# read query from sql file and execute
#query_1 <- getSQL("sql/data_pull__purchases.sql") 
#df_purchases <- dbGetQuery(con, query_1) %>% as_tibble()

#query_2 <- getSQL("sql/data_pull__purchases_same_size_by_size.sql") 
#df_purchases_same_size_by_size <- dbGetQuery(con, query_2) %>% as_tibble()

query_3 <- getSQL("sql/data_pull__purchases_same_size_overall.sql") 
df_purchases_same_size_overall <- dbGetQuery(con, query_3) %>% as_tibble()

query_4 <- getSQL("sql/data_pull__top-selling-designs.sql")
df_top_designs <- dbGetQuery(con, query_4) %>% as_tibble()

query_5 <- getSQL("sql/data_pull__rug_details.sql") 
df_rug_details <- dbGetQuery(con, query_5) %>% as_tibble()

options(warn=0)

In [18]:
df_rug_details <- df_rug_details %>% mutate(url = str_remove_all(url,"-two"))

In [19]:
df_purchases_same_size_overall <- df_purchases_same_size_overall %>% mutate(products_purchased = trim(products_purchased))

In [20]:
options(warn=-1)
#purchase_transactions <- makeTransaction(df_purchases)
#samesize_bysize_purchase_transactions <- makeTransaction(df_purchases_same_size_by_size)
samesize_overall_purchase_transactions <- makeTransaction(df_purchases_same_size_overall)
options(warn=0)

In [ ]:
num_top_rugs <- 200

In [21]:
options(warn=-1)
df_reco <- tibble(rug_name = character(),
                  reco_rug = character(),
                  confidence = numeric())
for (i in head(df_top_designs$rug_name,num_top_rugs)) {
    select_rug_rules <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.00001, conf=0.0001),appearance = list(lhs=i,default="rhs"))
    df <- as(subset(select_rug_rules,lift > 1.2), "data.frame")
    df_reco <- df_reco %>% add_row(rug_name = labels(lhs(subset(select_rug_rules,lift > 1.2))),
                                   reco_rug = labels(rhs(subset(select_rug_rules,lift > 1.2))),
                                   confidence = df$confidence)
}

df_reco <- df_reco %>% mutate(rug_name = str_remove_all(rug_name,"[{}]"),
                              reco_rug = str_remove_all(reco_rug,"[{}]"))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
      1e-04    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[790 item(s), 179283 transaction(s)] done [0.06s].
sorting and recoding items ... [788 item(s)] done [0.00s].
creating transaction tree ... done [0.05s].
checking subsets of size 1 2 done [0.00s].
writing ... [1302 rule(s)] done [0.00s].
creating S4 object  ... done [0.02s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
      1e-04    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose


In [1]:
num_recos <- 4

In [22]:
df_json <- df_reco %>% 
    inner_join(df_rug_details, by = c("reco_rug" = "rug_name")) %>%
    group_by(rug_name) %>%
    mutate(reco_rank = row_number(desc(confidence))) %>%
    filter(reco_rank <=num_recos) %>%
    select(rug_name, reco_rank, reco_rug, image__src, url) %>%
    arrange(rug_name, reco_rank) %>%
    pivot_wider(names_from = reco_rank, values_from = c(reco_rug, image__src, url)) %>%
    relocate(RugName = rug_name,
           Reco_RugName_1 = reco_rug_1,
           Reco_Image_1 = image__src_1,
           Reco_URL_1 = url_1,
           Reco_RugName_2 = reco_rug_2,
           Reco_Image_2 = image__src_2,
           Reco_URL_2 = url_2,
           Reco_RugName_3 = reco_rug_3,
           Reco_Image_3 = image__src_3,
           Reco_URL_3 = url_3,
           Reco_RugName_4 = reco_rug_4,
           Reco_Image_4 = image__src_4,
           Reco_URL_4 = url_4
          )

In [23]:
jsonData <- toJSON(x = df_json, dataframe = 'rows', pretty = T)

In [24]:
write(jsonData, "json-outputs/best-sellers-recommended-rugs-v20210806.json")

In [25]:
jsonData

[
  {
    "RugName": "Absida Rainbow Rug",
    "Reco_RugName_1": "Absida Teal Rug",
    "Reco_Image_1": "https://cdn.shopify.com/s/files/1/1033/0751/products/absida-teal-A-RC-MA003-57-V2.jpg?v=1610741355",
    "Reco_URL_1": "https://ruggable.com/products/absida-teal-rug?size=5x7",
    "Reco_RugName_2": "Metro Multicolor Rug",
    "Reco_Image_2": "https://cdn.shopify.com/s/files/1/1033/0751/products/metro-multicolor-A-RC-0735-57-V2.jpg?v=1612558753",
    "Reco_URL_2": "https://ruggable.com/products/metro-multicolor-rug?size=5x7",
    "Reco_RugName_3": "Floral Medallion Multicolor Rug",
    "Reco_Image_3": "https://cdn.shopify.com/s/files/1/1033/0751/products/floral-medallion-multicolor-A-RC-0296-57.jpg?v=1592350170",
    "Reco_URL_3": "https://ruggable.com/products/floral-medallion-multicolor?size=5x7",
    "Reco_RugName_4": "Gradasi Teal Quartz Rug",
    "Reco_Image_4": "https://cdn.shopify.com/s/files/1/1033/0751/products/gradasi-teal-quartz-A-RC-MA006-57-V2.jpg?v=1612388811",
    "Re